In [87]:
import numpy as np 
import pandas as pd 
import scipy as sipy 

In [88]:
df = pd.read_csv('heart.csv')

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [89]:
# print(df['restecg'].value_counts())
# اینجا این 4 تا ردیف رو دراپ کردم که دیگه به خاطر این 4 تا ردیف 3 ستون دیگه اضافه نشه 
# 1    152
# 0    147
# 2      4
# ----------------------------------------------

x = df[df['restecg'] != 2]
x = x[x['trestbps'] <= 175]
x = x[x['chol'] <= 350]

print(x)
x = df.iloc

y =  x[ : , -1 ].values
x = x[ : , : -1 ].values

x




     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
5     57    1   0       140   192    0        1      148      0      0.4   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  target  
0        0   0     1       1  
1        0   0     2     

array([[63.,  1.,  3., ...,  0.,  0.,  1.],
       [37.,  1.,  2., ...,  0.,  0.,  2.],
       [41.,  0.,  1., ...,  2.,  0.,  2.],
       ...,
       [68.,  1.,  0., ...,  1.,  2.,  3.],
       [57.,  1.,  0., ...,  1.,  1.,  3.],
       [57.,  0.,  1., ...,  1.,  1.,  2.]])

## Data Prepration : OHE for categorical and scale with standard scaler 

In [90]:
# OHE FOR cp-slope-ca-thal

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([(('cp','slope','ca','thal'),OneHotEncoder(),  [2, -3, -2, -1] )],remainder='passthrough')
x = ct.fit_transform(x)



In [91]:
# Standard Scaler

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
x = ss.fit_transform(x)

x[ : 3, : ]



array([[-0.94538352, -0.44455422, -0.63464776,  3.4891135 ,  3.66450153,
        -0.92676626, -0.93914194,  0.85523597, -0.52259855, -0.37867694,
        -0.26584089, -0.12953194, -0.08151391,  3.97911213, -1.10076284,
        -0.79311554,  0.9521966 ,  0.68100522,  0.76395577, -0.25633371,
         2.394438  , -1.00583187,  0.01544279, -0.69663055,  1.08733806],
       [-0.94538352, -0.44455422,  1.57567719, -0.28660575,  3.66450153,
        -0.92676626, -0.93914194,  0.85523597, -0.52259855, -0.37867694,
        -0.26584089, -0.12953194, -0.08151391, -0.25131234,  0.9084609 ,
        -0.79311554, -1.91531289,  0.68100522, -0.09273778,  0.07219949,
        -0.41763453,  0.89896224,  1.63347147, -0.69663055,  2.12257273],
       [-0.94538352,  2.24944438, -0.63464776, -0.28660575, -0.27288841,
        -0.92676626,  1.06480177,  0.85523597, -0.52259855, -0.37867694,
        -0.26584089, -0.12953194, -0.08151391, -0.25131234,  0.9084609 ,
        -0.79311554, -1.47415758, -1.46841752, -0

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)



algorithms = {
    'Decision Tree': (DecisionTreeClassifier(), {
        'max_depth': [3,5,7]
    }),
    'Random Forest': (RandomForestClassifier(), {
        'n_estimators': [50, 100, 200],
        'max_depth': [3,5,7],
        'min_samples_split': [2, 5, 10]
    })
}

for algorithm_name, (algorithm, params) in algorithms.items():
    grid_search = GridSearchCV(algorithm, params, cv=5, scoring='recall')
    grid_search.fit(X_train, y_train)

    print(f"best parameter for {algorithm_name}: {grid_search.best_params_}")
    print(f"best recall number for {algorithm_name}: {grid_search.best_score_}")
    print('-----------------------------------------------------------------------')

best_algorithm_name, (best_algorithm, best_params) = max(
    algorithms.items(), key=lambda x: grid_search.best_score_
)

best_model = best_algorithm.set_params(**best_params)
best_model.fit(X_train, y_train)

from sklearn.metrics import classification_report

y_pred = best_model.predict(X_test)
classification_rep = classification_report(y_test, y_pred)
print(classification_rep)

best parameter for Decision Tree: {'max_depth': 3}
best recall number for Decision Tree: 0.8504273504273504
-----------------------------------------------------------------------


best parameter for Random Forest: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50}
best recall number for Random Forest: 0.8871794871794872
-----------------------------------------------------------------------


InvalidParameterError: The 'max_depth' parameter of DecisionTreeClassifier must be an int in the range [1, inf) or None. Got [3, 5, 7] instead.

# 🌻 خسته نباشید 